In [ ]:
#
# --- 1. Setup: Clone Repository and Install Dependencies ---
#
print("--- Step 1: Cloning repository and installing project... ---")
!git clone https://github.com/DanielBerns/weather_forecast_at_the_cloud.git
%cd weather_forecast_at_the_cloud
# Install the project and its dependencies
!pip install .
print("--- Setup complete. ---")

In [ ]:
#
# --- 2. Configuration: Create Directories and config.yaml ---
#
import os
import yaml

print("--- Step 2: Creating directories and default config.yaml... ---")
# Create directories for processed data and models
os.makedirs("processed_data", exist_ok=True)
os.makedirs("models", exist_ok=True)

# Define the default configuration
# This is based on the config.yaml you provided.
# If your model or data uses different parameters, you can edit this cell.
config_data = {
    'model_name': 'LSTM',  # IMPORTANT: Change this to your model ('Linear', 'Dense', 'CNN', or 'LSTM')
    'epochs_per_session': 10,
    'sessions': 3,
    'patience': 3,
    'out_steps': 24,       # IMPORTANT: Must match your trained model
    'input_width': 24,     # IMPORTANT: Must match your trained model
    'conv_width': 3,
    'label_column': 'temperature_celsius', # IMPORTANT: Must match your trained model
    'raw_data_path': './raw_data',
    'processed_data_path': './processed_data',
    'models_path': './models',
    'results_path': './results',
    'logs_path': './logs',
    'gdrive_path': '/content/drive/MyDrive/weather_forecast_at_the_cloud',
}

# Write the configuration to config.yaml
config_path = "config.yaml"
with open(config_path, "w") as f:
    yaml.dump(config_data, f)

print(f"Created directories and default {config_path}.")
print("Please EDIT the 'model_name' in the cell above if it's not 'LSTM'.")

In [ ]:
#
# --- 3. Upload Files ---
#
from google.colab import files
import shutil

print("--- Step 3: Upload your files... ---")

# --- Upload test.csv ---
print("\n[Action] Please upload your processed 'test.csv' file:")
uploaded_files = files.upload()
if 'test.csv' in uploaded_files:
    shutil.move('test.csv', 'processed_data/test.csv')
    print("Moved 'test.csv' to 'processed_data/test.csv'")
else:
    print("ERROR: 'test.csv' not found in upload. Please re-run this cell.")

# --- Upload model files ---
print("\n[Action] Please upload your trained model files (e.g., 'LSTM.keras' and 'LSTM.json'):")
# We read the model name from the config to prompt the user
model_name = config_data.get('model_name', 'LSTM')
print(f"Expecting '{model_name}.keras' and '{model_name}.json'")
uploaded_model_files = files.upload()

# Move the uploaded model files
moved_model_files = False
for filename in uploaded_model_files.keys():
    if filename.endswith(('.keras', '.json')):
        shutil.move(filename, f'models/{filename}')
        print(f"Moved '{filename}' to 'models/{filename}'")
        moved_model_files = True

if not moved_model_files:
    print("ERROR: No model files (.keras or .json) found. Please re-run this cell.")

print("\n--- File upload complete. Verifying... ---")
!ls -l ./processed_data
!ls -l ./models

In [ ]:
#
# --- 4. Load Model, Data, and Visualize Predictions ---
#
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path

# Import the necessary classes from your project's source code
from weather_forecast_at_the_cloud.utils.window_generator import WindowGenerator
from weather_forecast_at_the_cloud.models.linear_weather_forecast import LinearWeatherForecast
from weather_forecast_at_the_cloud.models.dense_weather_forecast import DenseWeatherForecast
from weather_forecast_at_the_cloud.models.cnn_weather_forecast import CNNWeatherForecast
from weather_forecast_at_the_cloud.models.rnn_weather_forecast import RNNWeatherForecast

print("--- Step 4: Loading model and data for visualization... ---")

# --- 1. Load Config (already done in Cell 2, just reading it) ---
with open(config_path, "r") as f:
    config = yaml.safe_load(f)

model_name = config['model_name']
print(f"Loaded configuration for model: {model_name}")

# --- 2. Load the Trained Model ---
# This dictionary maps model names to their respective classes
models = {
    "Linear": LinearWeatherForecast,
    "Dense": DenseWeatherForecast,
    "CNN": CNNWeatherForecast,
    "LSTM": RNNWeatherForecast,
}

model_class = models.get(model_name)
if model_class is None:
    print(f"ERROR: Unknown model name: {model_name}")
else:
    # Base path for the model (e.g., './models/LSTM')
    model_base_path = Path(config['models_path']) / model_name

    print(f"Loading model from {model_base_path}...")
    # The .load() classmethod (from models/utils.py) handles suffixes
    try:
        model_wrapper = model_class.load(model_base_path)
        print("Model loaded successfully.")

        # --- 3. Load Test Data ---
        processed_data_path = Path(config['processed_data_path'])
        test_df_path = processed_data_path / 'test.csv'

        if not test_df_path.exists():
            print(f"ERROR: 'test.csv' not found at {test_df_path}")
        else:
            test_df = pd.read_csv(test_df_path, index_col='timestamp')
            # Ensure index is datetime for time-series plotting
            test_df.index = pd.to_datetime(test_df.index)
            print("Test data loaded.")

            # --- 4. Create Window Generator ---
            # A dummy DataFrame is needed to initialize the generator
            # since we only care about the .test property
            dummy_df = pd.DataFrame(columns=test_df.columns)

            test_window = WindowGenerator(
                input_width=config['input_width'],
                label_width=config['out_steps'],
                shift=config['out_steps'],
                train_df=dummy_df,  # Not needed for plotting
                val_df=dummy_df,    # Not needed for plotting
                test_df=test_df,
                label_columns=[config['label_column']]
            )

            print("WindowGenerator created for test data:")
            print(test_window)

            # --- 5. Plot Predictions vs. Actuals ---
            plot_column = config['label_column']
            print(f"\nGenerating plot for column: {plot_column}...")

            # Use the WindowGenerator's built-in plot method
            # It shows Inputs (blue), Actuals (green), and Predictions (orange)
            # We pass `model_wrapper.model` which is the actual Keras model
            test_window.plot(model_wrapper.model, plot_col=plot_column)
            plt.show()

    except FileNotFoundError:
        print(f"\n--- ERROR ---")
        print(f"Could not find model files at '{model_base_path}'.")
        print(f"Please make sure '{model_name}.keras' and '{model_name}.json' are in the 'models' folder.")
    except Exception as e:
        print(f"\nAn error occurred: {e}")